In [ ]:
import numpy as np
import torch
torch.set_default_dtype(torch.float64)

import h5py as h5

import tdg
import tdg.HMC as HMC
import tdg.plot as visualize

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
storage = 'analysis.h5'

Let's study a small, hot example for computational simplicity.

In [ ]:
nx = 5
lattice = tdg.Lattice(nx)

ere = tdg.EffectiveRangeExpansion(torch.tensor([1.0]))

tuning = tdg.AnalyticTuning(ere, lattice)
print(f'{tuning.C[0]}')

In [ ]:
nt = 8
beta = torch.tensor(1/25.)
mu = torch.tensor(-1.5*25.)
h  = torch.tensor([0,0,0], dtype=torch.float64)

S = tuning.Action(nt, beta, mu, h)

We could in principle use one Hamiltonian to do do the HMC Metropolis-Hastings accept/reject step and another to do the molecular dynamics integration.

Here we use the same Hamiltonian for both.

In [ ]:
H = HMC.Hamiltonian(S)
integrator = HMC.Omelyan(H, 20, 1)
hmc = HMC.MarkovChain(H, integrator)

Let's start from a configuration sampled from the "quenched" distribution, which ignores the fermion determinant.

In [ ]:
configurations = 1200

try:
    with h5.File(storage, 'r') as f:
        ensemble = tdg.ensemble.GrandCanonical.from_h5(f['/ensemble'])
    if len(ensemble) < configurations:
        raise
    
except:
    ensemble = tdg.ensemble.GrandCanonical(S).generate(configurations, hmc, start='hot', progress=tqdm)
    with h5.File(storage, 'w') as f:
        ensemble.to_h5(f.create_group('/ensemble'))

We can visualize an observable.

In [ ]:
viz = visualize.History(3)
viz.plot(ensemble.S.real, 0)
viz.plot(ensemble.N('bosonic').real, 1)
viz.plot(ensemble.Spin(0).real, 1) # Another way of calculating N('fermionic')
viz.plot(ensemble.Spin(1).real, 2)
viz.plot(ensemble.Spin(2).real, 2)
viz.plot(ensemble.Spin(3).real, 2)

# Binning

In [ ]:
binned = ensemble.cut(200).every(4).binned(4)

How many samples should be have, given that the ensemble started with 1200 configurations?

In [ ]:
expected = (
    (
        configurations-200 # cut
    )/4 # every
) / 4 # binning
print(f'The binning has {len(binned)} samples, while we expected {expected}')

Let's compare the binned samples with the original ensemble.

In [ ]:
viz = visualize.ScatterTriangle(2)
viz.plot(
    (ensemble.N('bosonic'  ).real,
     ensemble.N('fermionic').real
    ))
viz.plot(
    (binned.N('bosonic'  ).real,
     binned.N('fermionic').real
    ))

We can see how the binning averages samples across Markov chain time.

In [ ]:
viz = visualize.History(5)
viz.plot(ensemble.S.real, 0)
viz.plot(ensemble.N('bosonic').real, 1)
viz.plot(ensemble.Spin(0).real, 2) # Another way of calculating N('fermionic')
viz.plot(ensemble.Spin(1).real, 3)
viz.plot(ensemble.Contact('fermionic').real, 4)

viz.plot(binned.S.real, x=binned.index, row=0)
viz.plot(binned.N('bosonic').real, x=binned.index, row=1)
viz.plot(binned.Spin(0).real, x=binned.index, row=2) # Another way of calculating N('fermionic')
viz.plot(binned.Spin(1).real, x=binned.index, row=3)
viz.plot(binned.Contact('fermionic').real, x=binned.index, row=4)

# Bootstrapping

Let us compare different binnings and rest assured that our uncertainty estimate is big enough once the bootstrap errors stabilize.

In [ ]:
def naive_estimate(obs):
    return f'{obs.real.mean():.2f}±{obs.real.std()/torch.sqrt(torch.tensor(obs.shape[0])):.2f}'

def bootstrap_estimate(obs):
    return f'{obs.real.mean():.2f}±{obs.real.std():.2f}'

In [ ]:
print(f"ENSEMBLE")
# This obviously uses an extremely naive estimate for the uncertainty!
print(f"fermionic N: {naive_estimate(ensemble.N('fermionic'))}")
print(f"bosonic   N: {naive_estimate(ensemble.N('bosonic'))}")
print(f"action:      {naive_estimate(ensemble.S)}")
print(f"Contact:     {naive_estimate(ensemble.Contact('fermionic'))}\n")

viz = visualize.ScatterTriangle(4, figsize=(12,12), labels=('fermionic N', 'bosonic N', 'action', 'Contact',))
for width in [1, 2, 4, 8, 16, 24, 32]:
    bootstrap = ensemble.cut(200).binned(width).bootstrapped()
    viz.plot(
        (bootstrap.N('bosonic'  ).real,
         bootstrap.N('fermionic').real,
         bootstrap.S.real,
         bootstrap.Contact('fermionic').real,
        ))
    
    print(f"BOOTSTRAP {width=} ({len(binned)} bins)")
    print(f"fermionic N: {bootstrap_estimate(bootstrap.N('fermionic'))}")
    print(f"bosonic   N: {bootstrap_estimate(bootstrap.N('bosonic'))}")
    print(f"action:      {bootstrap_estimate(bootstrap.S)}")
    print(f"Contact:     {bootstrap_estimate(bootstrap.Contact('fermionic'))}\n")


We see that the uncertainties have stabilized: the binnings of widths 24 and 32 give approximately the same uncertainty.